In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#Load the libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

import os
print(os.listdir("../input"))
import warnings
warnings.filterwarnings('ignore')

In [3]:
imdb_data=pd.read_csv('../input/imdb-dataset/IMDB Dataset.csv')
print(imdb_data.shape)
imdb_data.head(10)

In [4]:
#summary of data
imdb_data.describe()

In [5]:
#Checking distribution of positive and negative sentiments
imdb_data['sentiment'].value_counts()

In [6]:
#splitting the dataset
#train dataset reviews and seniments
train_reviews_data=imdb_data.review[:40000]
train_sentiments_data=imdb_data.sentiment[:40000]
#test dataset reviews and seniments
test_reviews_data=imdb_data.review[40000:]
test_sentiments_data=imdb_data.sentiment[40000:]
print(train_reviews_data.shape,train_sentiments_data.shape)
print(test_reviews_data.shape,test_sentiments_data.shape)

In [7]:
tokenizer=ToktokTokenizer()
#Setting stopwords
stopword_list=nltk.corpus.stopwords.words('english')

In [8]:
#Removing the html strips and noise from the text
def remove_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    text = remove_html(text)
    text = remove_between_square_brackets(text)
    return text
#Applying this function on review column
imdb_data['review']=imdb_data['review'].apply(denoise_text)

In [9]:
#Removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text
#Applying this function on review column
imdb_data['review']=imdb_data['review'].apply(remove_special_characters)

In [10]:
#Stemming
def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text
#Applying stemming on review column
imdb_data['review']=imdb_data['review'].apply(simple_stemmer)

In [11]:
#Removing stop words
stop=set(stopwords.words('english'))

#Function for removing stopwords from text
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text


#Removing stop words from review column
imdb_data['review']=imdb_data['review'].apply(remove_stopwords)

In [12]:
#Normalizing test and Train Data
#imdb_data.review[:40000]
norm_train_reviews=imdb_data.review[:40000]
norm_train_reviews[0]
#imdb_data.review[40000:]
norm_test_reviews=imdb_data.review[40000:]
norm_test_reviews[45005]

In [13]:
#using Count vectorizer for bag of words model
#cv=CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))
#transformed train reviews
#cv_train_reviews=cv.fit_transform(norm_train_reviews)
#transformed test reviews
#cv_test_reviews=cv.transform(norm_test_reviews)

#print('BOW_cv_train:',cv_train_reviews.shape)
#print('BOW_cv_test:',cv_test_reviews.shape)

In [14]:
#Term Frequency-Inverse Document Frequency model (TFIDF)
tv=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
#transformed train reviews
tv_train_reviews=tv.fit_transform(norm_train_reviews)
#transformed test reviews
tv_test_reviews=tv.transform(norm_test_reviews)
print('Tfidf_train:',tv_train_reviews.shape)
print('Tfidf_test:',tv_test_reviews.shape)

In [15]:
#labelling

lb=LabelBinarizer()
#transformed sentiment data
sentiment_data=lb.fit_transform(imdb_data['sentiment'])
print(sentiment_data.shape)
#Spliting the sentiment data
train_sentiments=sentiment_data[:40000]
test_sentiments=sentiment_data[40000:]
print(train_sentiments)
print(test_sentiments)

In [16]:
#Logistic regression model
#training the model
lr=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=42)
#Fitting the model for tfidf features
lr_tfidf=lr.fit(tv_train_reviews,train_sentiments)
print(lr_tfidf)

##Predicting the model for tfidf features
lr_tfidf_predict=lr.predict(tv_test_reviews)

In [17]:
#Accuracy score for tfidf features
lr_tfidf_score=accuracy_score(test_sentiments,lr_tfidf_predict)
print("lr_tfidf_score :",lr_tfidf_score)

In [18]:
#Classification report 
lr_tfidf_report=classification_report(test_sentiments,lr_tfidf_predict,target_names=['Positive','Negative'])
print(lr_tfidf_report)

In [19]:
#confusion matrix for tfidf features
cm_tfidf=confusion_matrix(test_sentiments,lr_tfidf_predict,labels=[1,0])
print(cm_tfidf)

In [20]:
#training the linear svm
svm=SGDClassifier(loss='hinge',max_iter=500,random_state=42)
svm_tfidf=svm.fit(tv_train_reviews,train_sentiments)

In [21]:
svm_tfidf_predict=svm.predict(tv_test_reviews)
svm_tfidf_score=accuracy_score(test_sentiments,svm_tfidf_predict)
print("svm_tfidf_score :",svm_tfidf_score)

In [22]:
#Multinomial Naive Bayes for bag of words and tfidf features
#training the model
mnb=MultinomialNB()
mnb_tfidf=mnb.fit(tv_train_reviews,train_sentiments)
#Predicting the model for tfidf features
mnb_tfidf_predict=mnb.predict(tv_test_reviews)


In [23]:
mnb_tfidf_score=accuracy_score(test_sentiments,mnb_tfidf_predict)
print("mnb_tfidf_score :",mnb_tfidf_score)